# Presentielle 2017 Résultats Tour 1 

In [1]:
import pandas as pd

## 1) Import Fichier Excel

In [2]:
excel_tour1 = pd.ExcelFile('Presidentielle_2017_Resultats_Tour_1_md.xls')
coordonnes = pd.ExcelFile('Coordonnees.xlsx')

# Région

In [3]:
# Eléction
#---------

dataframe_region=pd.read_excel(excel_tour1,sheet_name = [1])
# dataframe_region

# Coordonneées
#-------------

dataframe_coordonnes_regions=pd.read_excel(coordonnes,sheet_name=[0])

centre_lat=dataframe_coordonnes_regions[0]['centre_lat']
centre_lan=dataframe_coordonnes_regions[0]['centre_lan']

nom_region=dataframe_coordonnes_regions[0]['Libellé de la région']
code_region = dataframe_coordonnes_regions[0]['Code de la région']

In [4]:
le_plus_vote=dataframe_region[1][['Code de la région','Libellé de la région','Inscrits','Votants','Sexe','Nom','Prénom','Voix','% Voix/Ins']]

In [114]:
le_plus_vote.to_dict()

{'Code de la région': {0: 44,
  1: 75,
  2: 84,
  3: 27,
  4: 53,
  5: 24,
  6: 11,
  7: 76,
  8: 32,
  9: 28,
  10: 52,
  11: 93,
  12: 94,
  13: 1,
  14: 2,
  15: 3,
  16: 4,
  17: 6},
 'Libellé de la région': {0: 'Grand Est',
  1: 'Nouvelle-Aquitaine',
  2: 'Auvergne et Rhône-Alpes',
  3: 'Bourgogne-Franche-Comté',
  4: 'Bretagne',
  5: 'Centre-Val de Loire',
  6: 'Île-de-France',
  7: 'Occitanie',
  8: 'Hauts-de-France',
  9: 'Normandie',
  10: 'Pays de la Loire',
  11: "Provence-Alpes-Côte d'Azur",
  12: 'Corse',
  13: 'Guadeloupe',
  14: 'Martinique',
  15: 'Guyane',
  16: 'La Réunion',
  17: 'Mayotte'},
 'Inscrits': {0: 3873402,
  1: 4329942,
  2: 5416712,
  3: 2001656,
  4: 2453683,
  5: 1827657,
  6: 7199231,
  7: 4194294,
  8: 4241887,
  9: 2398182,
  10: 2705170,
  11: 3567740,
  12: 233656,
  13: 316345,
  14: 310812,
  15: 92935,
  16: 639481,
  17: 83047},
 'Votants': {0: 3047571,
  1: 3483794,
  2: 4297161,
  3: 1588659,
  4: 2048612,
  5: 1466892,
  6: 5751509,
  7: 339

In [7]:
plus_de_20 = le_plus_vote[le_plus_vote["% Voix/Ins"]>20]

In [8]:
plus_de_20

,Code de la région,Libellé de la région,Inscrits,Votants,Sexe,Nom,Prénom,Voix,% Voix/Ins
0,44,Grand Est,3873402,3047571,F,LE PEN,Marine,825604,21.31
4,53,Bretagne,2453683,2048612,M,MACRON,Emmanuel,581076,23.68
6,11,Île-de-France,7199231,5751509,M,MACRON,Emmanuel,1612516,22.40
8,32,Hauts-de-France,4241887,3316694,F,LE PEN,Marine,1003216,23.65
10,52,Pays de la Loire,2705170,2252699,M,MACRON,Emmanuel,575832,21.29
11,93,Provence-Alpes-Côte d'Azur,3567740,2810838,F,LE PEN,Marine,774781,21.72


# Folium

In [29]:
import numpy as np
import json
import requests
import branca
import folium
import branca.colormap as cmp

# ################ #
#  France (Région) #
# ################ #

# 1) url
url = 'https://france-geojson.gregoiredavid.fr'
region_geo= f'{url}/repo/regions.geojson'

# 2) Folium
region_map = folium.Map(location=[46.91160617052484, 2.153649265809747]
                       , zoom_start=5.5)

def colorByName(Nom):
    name_dict = {'MACRON' : '#d45087',
                                     'MÉLENCHON' : '#665191',
                                     'LE PEN' : '#003f5c',
                                     'HAMON' : 'yellow',
                                     'DUPONT' : 'grey',
                                     'ARTHAUD' : '#f95d6a',
                                     'POUTOU' : '#003f5c', 
                                     'CHEMINADE' : '#ff7c43', 
                                     'LASSALLE' : '#ffa600',
                                     'ASSELINEAU' : '#a05195',
                                     'FILLON' : '#2f4b7c'}
    
    return name_dict[Nom]

def fillColor(feature):
    code = int(feature['properties']['code'])
    code_name=feature['properties']['nom']
    candidate_name=le_plus_vote[le_plus_vote["Code de la région"]==code]['Nom'].to_string(index=False)
    
    return colorByName(candidate_name)


folium.GeoJson(
        region_geo,
        style_function= lambda feature: {
        'fillColor' : fillColor(feature),
        'color': 'white',
        'weight' : 1,
        'opacity' : 1,
        'fillOpacity':0.8
        }
).add_to(region_map)


for i in range(len(dataframe_coordonnes_regions[0])):
    
    html="""<h4>""" + nom_region[i] + """</h4>
        <iframe src=\"resultats/resultats_"""+str(i)+""".html\" width="600" height="450"  frameborder="0">    

        """
    folium.Marker(location=[centre_lat[i],centre_lan[i]],
                  popup = folium.Popup(html, min_width=300, max_width=600),
                  icon= folium.Icon(color='white',
                  icon_color='#0b0d17',icon="pie-chart", prefix='fa')).add_to(region_map)

region_map

# HTML SAVE
#region_map.save("region.html")


In [10]:
from pandas import json_normalize

In [11]:
# json_normalize(pd.read_json(region_geo).features)['properties.code']
json_normalize(pd.read_json(region_geo).features)['geometry.coordinates'][0]

[[[1.9221462784913, 48.457599361977],
  [1.9207376717625, 48.447750046159],
  [1.9065549813045002, 48.445672344539],
  [1.9064076592587, 48.440146719021],
  [1.8774887068591002, 48.440613561663],
  [1.8448630603718001, 48.449360706547],
  [1.8363839553347, 48.466477806902],
  [1.8014512893542, 48.466086215218],
  [1.7963309289573002, 48.484187938277],
  [1.7857092863217, 48.489936911612],
  [1.7906352036641, 48.497377315443],
  [1.7786170432038, 48.508131705459],
  [1.7757543018107, 48.527702827198],
  [1.7872423006556, 48.553746336139],
  [1.7674104287362, 48.559405546059],
  [1.765322855183, 48.569374568412],
  [1.7458629248393, 48.576137031624],
  [1.7368302606076, 48.572280930629],
  [1.7092486767776, 48.578026100992],
  [1.701983609829, 48.584999462088],
  [1.7178604312421, 48.606851120102],
  [1.7148984609516, 48.614392276576],
  [1.6894221050990001, 48.611536250362],
  [1.6793591385148, 48.618488989615],
  [1.6662229993646, 48.613704063337],
  [1.656339110342, 48.622053121145],


In [47]:
region_map3 = folium.Map(location=[46.91160617052484, 2.153649265809747]
                       , zoom_start=5.5)
folium.GeoJson(
        region_geo,
        style_function= lambda feature: {
        'fillColor' : 'yellow',
        'color': 'black',
        'weight' : 1,
        'opacity' : 1,
        'fillOpacity':0.4
        }
).add_to(region_map3)


for i in range(len(dataframe_coordonnes_regions[0])):
    
    html="""<h2>""" + nom_region[i] + """</h2>
        """
        
    
    folium.Marker(location=[centre_lat[i],centre_lan[i]],
                  popup = folium.Popup(html, min_width=300, max_width=300),
                  icon= folium.Icon(color='red',
                  icon_color='yellow',icon = 'cloud')).add_to(region_map3)

In [48]:
region_map3